# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

## Deadlines ([AoE](https://en.wikipedia.org/wiki/Anywhere_on_Earth))

* Sunday, December 4th for Task3.1 and Task3.2
* Sunday, December 11th for Task3.3 and Task3.4
* Sunday, December 18th for all reviews

In [828]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

## The *Nim* and *Nimply* classes

In [829]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [830]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [831]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [832]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [833]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()


    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    #cooked["nim_sum"] = nim_sum(state)
    cooked["avg_objects"] = sum(i for i in state.rows)/len(state.rows)
    cooked["over_avg_rows"] = list(i for i, e in enumerate(state.rows) if e >= cooked["avg_objects"])
    cooked["under_avg_rows"] = list(i for i, e in enumerate(state.rows) if e <= cooked["avg_objects"] and e != 0)

    if cooked["active_rows_number"]/len(state.rows) > 0.8:
        cooked["game_type"] = "early"
    elif cooked["active_rows_number"]/len(state.rows) > 0.2:
        cooked["game_type"] = "mid"
    else:
        cooked["game_type"] = "end"

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force


    return cooked

In [834]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [835]:
def strange_strategy(state: Nim) -> Nimply:
    cooked = cook_status(state)
    if abs(cooked["avg_objects"] - state.rows[cooked["longest_row"]]) < abs(cooked["avg_objects"] - state.rows[cooked["shortest_row"]]) or len(cooked["under_avg_rows"]) == 0:
        row = random.choice(cooked["over_avg_rows"])
        num_objects = random.randint(1, state.rows[row])
    else:
        row = random.choice(cooked["under_avg_rows"])
        num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [836]:
def strategy_v2(state: Nim) -> Nimply:
    cooked = cook_status(state)
    if cooked["active_rows_number"] % 3 == 0:
        row = random.choice([i for i, e in enumerate (state.rows) if e > 0])
        num_objects = state.rows[row]

    elif cooked["active_rows_number"] % 2 == 0:
        if state.rows[cooked["longest_row"]] > 1:
            row = random.choice([i for i, e in enumerate (state.rows) if e > 1])
            num_objects = state.rows[row]-1
        else:
            row = random.choice([i for i, e in enumerate (state.rows) if e > 0])
            num_objects = state.rows[row]

    else:
        row = random.choice([i for i, e in enumerate (state.rows) if e > 0])
        #num_objects = random.randint(1, max(1, state.rows[row]-1))
        num_objects = state.rows[row]

    return Nimply(row, num_objects)

In [837]:
def strategy_v3(state: Nim) -> Nimply:
    cooked = cook_status(state)

    if cooked["active_rows_number"] % 2 == 1:
        row = random.choice([i for i, e in enumerate (state.rows) if e > 0])
        num_objects = state.rows[row]

    else:
        if state.rows[cooked["longest_row"]] > 1:
            row = random.choice([i for i, e in enumerate (state.rows) if e > 1])
            num_objects = state.rows[row]-1
        else:
            row = random.choice([i for i, e in enumerate (state.rows) if e > 0])
            num_objects = state.rows[row]

    return Nimply(row, num_objects)

In [838]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

In [839]:
NUM_MATCHES = 1000
NIM_SIZE = 10


def evaluate(strategy: Callable) -> float:
    opponent = (strategy, pure_random)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

In [840]:
evaluate(strategy_v3)

0.921

## Oversimplified match

In [841]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (strategy_v3, pure_random)

nim = Nim(10)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19>
DEBUG:root:status: After player 0 -> <1 3 5 1 9 11 13 15 17 19>
DEBUG:root:status: After player 1 -> <1 3 5 0 9 11 13 15 17 19>
DEBUG:root:status: After player 0 -> <1 3 5 0 9 11 13 15 0 19>
DEBUG:root:status: After player 1 -> <1 2 5 0 9 11 13 15 0 19>
DEBUG:root:status: After player 0 -> <1 2 5 0 9 11 1 15 0 19>
DEBUG:root:status: After player 1 -> <0 2 5 0 9 11 1 15 0 19>
DEBUG:root:status: After player 0 -> <0 2 0 0 9 11 1 15 0 19>
DEBUG:root:status: After player 1 -> <0 2 0 0 9 11 1 15 0 16>
DEBUG:root:status: After player 0 -> <0 1 0 0 9 11 1 15 0 16>
DEBUG:root:status: After player 1 -> <0 1 0 0 9 11 1 15 0 14>
DEBUG:root:status: After player 0 -> <0 1 0 0 1 11 1 15 0 14>
DEBUG:root:status: After player 1 -> <0 1 0 0 1 11 0 15 0 14>
DEBUG:root:status: After player 0 -> <0 1 0 0 1 0 0 15 0 14>
DEBUG:root:status: After player 1 -> <0 1 0 0 0 0 0 15 0 14>
DEBUG:root:status: After player 0 -> <0 1 0 0 0 0 0 15 0 0>
DEBU